In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import time
import os
from copy import deepcopy
import math
import random
from sklearn.metrics import root_mean_squared_error
from scipy.stats import ttest_ind
import pickle
import ast
import json



from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [3]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block
def get_xmap_results_smap(block, target, embeddings, Tp, theta, lib, pred):
    '''Function to do exhaustive search of embeddings.'''
    
    def compute_rho(block, target, embedding, Tp, theta, lib, pred):
        xmap = SMap(dataFrame=block, target=target, columns=embedding, Tp=Tp, theta=theta, embedded=True, lib=lib, pred=pred, noTime=True)
        rho = xmap['predictions'][['Observations', 'Predictions']].corr().iloc[0,1]
        return embedding, xmap['predictions'], rho

    xmap_results = pd.DataFrame(columns=['embedding', 'rho'])
    xmap_results = Parallel(n_jobs=-1)(delayed(compute_rho)(block, target, embedding, Tp, theta, lib, pred) for embedding in embeddings)
    xmap_results = pd.DataFrame(xmap_results, columns=['embedding', 'result', 'rho'])
    xmap_results = xmap_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    return xmap_results

def get_valid_lags_tau(block, target, tau, num_lags, system_variables):
    
    # Get lags of system variables
    system_variable_lags = []
    for var in system_variables:
        var_lags = [f'{var}(t{i})' if i < 0 else f'{var}(t-{i})' for i in range(num_lags * tau, 0)]
        var_lags = var_lags[::tau][:num_lags]
        system_variable_lags = system_variable_lags + var_lags
    
    # Remove (t-1) lag of target variable from valid_lags
    valid_lags = [x for x in system_variable_lags if x[-4:-1] != f'{target}(t-1)']

    print(f'Valid lags are {valid_lags}')        
    return valid_lags


def create_single_model(E,theta,target,i_cols,lib, pred,HAB_embed,showPlot=False):
    driver = f'{target}(t-0)'
    #print(i_cols)
    if not(f'{target}(t-1)' in i_cols):
        cols = i_cols + [f'{target}(t-1)']
    else:
        cols = i_cols
    #print(cols)
    result = SMap(
        dataFrame = HAB_embed, 
        columns = cols,
        target = driver,
        lib = lib,  # Library from rows 0 to 700
        pred = pred,
        E = E+1,
        theta=theta,
        noTime=True,
        showPlot = showPlot,
        embedded=True,
        ignoreNan = True
    )
    return result

def thresh_bloom_binary_prediction(obs,pred,threshold=8.03199999999999):
    #obs_bloom_95 = np.percentile(obs, 95) #incorrect
    #pred_bloom_95 = np.percentile(pred, 95) #incorrect
    obs_blooms = obs > threshold
    pred_blooms = pred > threshold
    Accuracy = 1 - (obs_blooms ^ pred_blooms).mean()
    True_pos = (obs_blooms & pred_blooms).sum() / obs_blooms.sum()
    False_pos = ((~obs_blooms) & pred_blooms).sum() / (~obs_blooms).sum()
    True_neg = ((~obs_blooms) & (~pred_blooms)).sum() / (~obs_blooms).sum()
    False_neg = (obs_blooms & (~pred_blooms)).sum() / obs_blooms.sum()
    
    return [Accuracy, True_pos, False_pos, True_neg, False_neg]


def create_model(data,system_variables,target,Tp,num_lags,tau,theta_list,lib,pred):
    HAB_embed_block = get_block(data,50)
    valid_lags = get_valid_lags_tau(HAB_embed_block, target, tau, num_lags, system_variables)
    HAB_embeddings = {}
    for E in range(3,11):
        # Get random embeddings using valid lags
        print(f'Creating models for E = {E}')
        embeddings = set()
        sample = 1000
        max_trials = 1000
        trials = 0
        while len(embeddings) < sample and trials < max_trials:
            embedding = tuple(random.sample(valid_lags, E))
            sorted_embedding = tuple(sorted(embedding))
            if sorted_embedding not in embeddings:
                embeddings.add(sorted_embedding)
            trials += 1
        embeddings = [list(embedding) for embedding in embeddings]
        HAB_embeddings['{0}'.format((target, E))] = embeddings
        E_list = range(3,11)


        
    total_iterations = len(E_list) * len(theta_list)

    parameters = pd.DataFrame(columns=['target', 'columns', 'E', 'theta', 'rho', 'rmse', 'pred'])

    with tqdm(total=total_iterations) as pbar:
        for E, theta in product(E_list, theta_list):

            key = [key for key in HAB_embeddings.keys() if eval(key)[0] == target and eval(key)[1] == E] #HAB_embeddings["('Avg_Chloro', 4, 0, 6)"][0]
            embeddings = HAB_embeddings[key[0]]
            for embedding in embeddings:
                smap_model = create_single_model(E,theta,target,embedding,lib, pred,HAB_embed_block,showPlot=False)
                df = smap_model['predictions']
                rho = df[['Observations', 'Predictions']].corr().iloc[0,1]
                rmse = root_mean_squared_error(df['Observations'].iloc[1:-1], df['Predictions'].iloc[1:-1])
                #bbp = thresh_bloom_binary_prediction(df['Observations'].iloc[1:-1],df['Predictions'].iloc[1:-1])

                new_row = {'target': target, 'columns': embedding + [f'{target}(t-1)'], 'E': E,'theta':theta, 'rho':rho, 'rmse':rmse, 'pred':df['Predictions']}
                parameters.loc[len(parameters)] = new_row

            

            pbar.update(1)
    parameters.sort_values(by='rho',ascending=False)
    return parameters

In [4]:
def ensemble_binary_bloom(parameters_df,n=300,p=0.05,samp=1,bloom_thresh=8.013):
    parameters_df = parameters_df.iloc[:n*samp].sample(n)
    sum = np.zeros(np.array(parameters_df['pred'].iloc[0][1:]).size)
    for i in range(n):
        curr = np.array(parameters_df['pred'].iloc[i][1:]) > bloom_thresh#np.percentile(parameters_df['pred'].iloc[i].iloc[1:],95)#
        sum = sum + curr
    return sum > (n*p)

'''
@parameters
data - dataframe of data containing column for target and desired system variables'
system variables - variables contained in system to be used for prediction
target - variable that will be forecasted (also used in prediction)
Tp - IDK
E - number of lags we go back
tau - step size that we go back at
theta_list - list of theta values to be used in ensemble model
lib - library used for prediction
pred - prediction length

@return
returns forecast for next time step given the dataframe 
'''

def next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred,p=0.05):
    parameters = create_model(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
    return parameters
    


# Testings

In [5]:

paper_data = pd.read_csv('Data/carter_data_w_gaps.csv') #DESIRED DATAPATH

paper_data = paper_data.set_index('time')
paper_data['Time'] = paper_data.index.astype(int)
paper_data['Avg_Chloro'] #= paper_data['Avg_Chloro'].apply(np.log1p) #LOG AMPUTATION
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
paper_data = paper_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(paper_data)
paper_data = Imputer.transform(paper_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
paper_data
HAB_embed = get_block(paper_data,50)
HAB_embed

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_15594/1653864451.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_15594/1653864451.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


,Unnamed: 0(t-0),Akashiwo_sanguinea(t-0),CellCountDetection_Limit(t-0),Lingulodinium_polyedra(t-0),Total_Diatoms(t-0),Total_Dinoflagellates(t-0),Total_Phytoplankton(t-0),Ammonium(t-0),Avg_Chloro(t-0),Avg_Phaeo(t-0),...,Total_Prorocentrum_spp(t-50),Total_Tripos(t-50),WSPD(t-50),index(t-50),SURF_DENS_kgm3(t-50),BOT_DENS_kgm3(t-50),AVG_TEMP_C(t-50),AVG_SAL_PSU(t-50),AVG_DENS_kgm3(t-50),Time(t-50)
time,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,89.0,89.0,69952.0,15684.0,85636.0,0.530,1.82,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,44.5,89.0,89.0,118740.5,10693.5,129434.5,0.795,2.08,0.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,89.0,89.0,89.0,167529.0,5703.0,173233.0,1.060,2.71,1.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,89.0,89.0,178.0,117315.0,8109.0,125424.5,0.705,1.32,0.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,89.0,89.0,267.0,67101.0,10515.0,77616.0,0.350,0.99,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,1597.0,0.0,626.0,0.0,95122.0,63206.0,158328.0,0.410,3.38,1.00,...,64458.0,0.0,1.965278,1547.0,1023.943315,1024.037281,17.75,33.370,1023.990298,1547.0
1598,1598.0,0.0,626.0,5006.5,191182.5,114209.0,305391.5,0.410,6.75,0.88,...,28787.0,1252.0,5.076389,1548.0,1024.146195,1024.152950,17.40,33.470,1024.149572,1548.0
1599,1599.0,0.0,626.0,10013.0,287243.0,165212.0,452455.0,0.410,9.71,1.49,...,26909.5,626.0,3.731250,1549.0,1023.909736,1023.973271,18.40,33.515,1023.941504,1549.0


In [6]:
data = paper_data

system_variables = ['Avg_Chloro','Akashiwo_sanguinea', 'Avg_Phaeo', 'CellCountDetection_Limit', 'Lingulodinium_polyedra', 'Nitrate', 'Phaeo1', 'Phaeo2', 'Silicate', 'Total_Diatoms', 'Total_Dinoflagellates', 'Total_Phytoplankton'] #SYSTEM VARIABLES



target = 'Avg_Chloro' #TARGET
system_variables = system_variables
Tp = 1
E = 6 # up to -5 
tau = -1
theta_list = [1,5,9,15,25,45]
lib = '1 1012'
pred = '1013 1602'


preds = next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
preds

Valid lags are ['Avg_Chloro(t-1)', 'Avg_Chloro(t-2)', 'Avg_Chloro(t-3)', 'Avg_Chloro(t-4)', 'Avg_Chloro(t-5)', 'Avg_Chloro(t-6)', 'Akashiwo_sanguinea(t-1)', 'Akashiwo_sanguinea(t-2)', 'Akashiwo_sanguinea(t-3)', 'Akashiwo_sanguinea(t-4)', 'Akashiwo_sanguinea(t-5)', 'Akashiwo_sanguinea(t-6)', 'Avg_Phaeo(t-1)', 'Avg_Phaeo(t-2)', 'Avg_Phaeo(t-3)', 'Avg_Phaeo(t-4)', 'Avg_Phaeo(t-5)', 'Avg_Phaeo(t-6)', 'CellCountDetection_Limit(t-1)', 'CellCountDetection_Limit(t-2)', 'CellCountDetection_Limit(t-3)', 'CellCountDetection_Limit(t-4)', 'CellCountDetection_Limit(t-5)', 'CellCountDetection_Limit(t-6)', 'Lingulodinium_polyedra(t-1)', 'Lingulodinium_polyedra(t-2)', 'Lingulodinium_polyedra(t-3)', 'Lingulodinium_polyedra(t-4)', 'Lingulodinium_polyedra(t-5)', 'Lingulodinium_polyedra(t-6)', 'Nitrate(t-1)', 'Nitrate(t-2)', 'Nitrate(t-3)', 'Nitrate(t-4)', 'Nitrate(t-5)', 'Nitrate(t-6)', 'Phaeo1(t-1)', 'Phaeo1(t-2)', 'Phaeo1(t-3)', 'Phaeo1(t-4)', 'Phaeo1(t-5)', 'Phaeo1(t-6)', 'Phaeo2(t-1)', 'Phaeo2(t-2)', 

100%|██████████| 48/48 [2:14:33<00:00, 168.19s/it]


,target,columns,E,theta,rho,rmse,pred
0,Avg_Chloro,"[Avg_Phaeo(t-6), Total_Diatoms(t-5), Total_Din...",3,1,0.428537,56.292059,0 NaN 1 1.384817 2 1.47...
1,Avg_Chloro,"[Nitrate(t-6), Phaeo1(t-6), Phaeo2(t-3), Avg_C...",3,1,0.341186,59.274036,0 NaN 1 1.362143 2 1.56...
2,Avg_Chloro,"[Akashiwo_sanguinea(t-5), Nitrate(t-3), Nitrat...",3,1,0.337825,59.832347,0 NaN 1 1.585193 2 1.66...
3,Avg_Chloro,"[Avg_Phaeo(t-3), Lingulodinium_polyedra(t-2), ...",3,1,0.411991,56.946754,0 NaN 1 1.520982 2 1.62...
4,Avg_Chloro,"[Phaeo2(t-2), Silicate(t-1), Silicate(t-5), Av...",3,1,0.338021,59.654946,0 NaN 1 1.276348 2 1.55...
...,...,...,...,...,...,...,...
47941,Avg_Chloro,"[Akashiwo_sanguinea(t-4), Avg_Chloro(t-3), Avg...",10,45,0.041553,115.175955,0 NaN 1 0.830443 2 1.34...
47942,Avg_Chloro,"[CellCountDetection_Limit(t-3), CellCountDetec...",10,45,0.300227,587.684139,0 NaN 1 1.164812 2 1.21763...
47943,Avg_Chloro,"[Akashiwo_sanguinea(t-5), Avg_Chloro(t-1), Cel...",10,45,-0.203324,395.404099,0 NaN 1 1.309853 2 0.91651...
47944,Avg_Chloro,"[Akashiwo_sanguinea(t-1), Avg_Phaeo(t-5), Cell...",10,45,-0.001221,320.701566,0 NaN 1 1.044545 2 2.80...


In [9]:
preds['pred'].iloc[0]

[nan,
 1.3848168369414413,
 1.4709325693175295,
 2.1750252316627057,
 2.0501520705367478,
 1.4887020011799867,
 1.205573727496803,
 1.2578980010347356,
 1.2089564067184788,
 1.6244374770716896,
 1.6590495523110693,
 1.7415105554700103,
 1.4309399974441963,
 1.7153419215566408,
 1.8566447892074633,
 2.6793096858682235,
 3.074260233587079,
 1.591180484836256,
 2.1039131189690483,
 2.071423408578523,
 1.9736497865807183,
 1.8857787314902892,
 2.502124966062411,
 2.3450758105861724,
 2.2063394267898904,
 1.545215433432799,
 2.1994310727180206,
 2.9905432221057975,
 2.5718323633517297,
 2.9341208890380663,
 2.446220490499766,
 3.66342895495535,
 2.981141802461872,
 2.6036514359631466,
 1.43742154401443,
 1.326888159296686,
 1.9301691221841875,
 1.8271927999673445,
 1.7436737490574883,
 1.6234186849054681,
 2.514427993443512,
 3.659762117094722,
 6.677519563024819,
 11.231662293114699,
 5.648039418512024,
 8.07969636140744,
 36.96003780265409,
 17.01436359683891,
 5.024954787313218,
 7.89356

In [8]:
preds['pred'] = preds['pred'].apply(list)
preds.to_csv('1000_sample_HAB_cor_carter_models.csv') #TO SAVE (BE CAREFUL ABOUT OVERWRITING)

In [119]:
preds.sort_values(by='rho',ascending=False)

,target,columns,E,theta,rho,rmse,pred
11684,Avg_Chloro,"[Avg_Phaeo(t-1), Lingulodinium_polyedra(t-1), ...",4,45,0.903767,316.675203,"[nan, 1.4896604175248014, 1.3929536645039562, ..."
47245,Avg_Chloro,"[Akashiwo_sanguinea(t-1), Avg_Phaeo(t-1), Ling...",10,45,0.899577,382.361426,"[nan, 0.9615772402434983, 1.050903510251775, 2..."
23884,Avg_Chloro,"[Lingulodinium_polyedra(t-1), Lingulodinium_po...",6,45,0.893186,467.056724,"[nan, 1.5056883753722687, 1.2868716973039, 1.2..."
23223,Avg_Chloro,"[Avg_Phaeo(t-1), Lingulodinium_polyedra(t-1), ...",6,45,0.892362,417.203049,"[nan, 1.052760117470292, 1.1455392193112566, 1..."
17743,Avg_Chloro,"[Avg_Phaeo(t-1), Phaeo1(t-3), Total_Diatoms(t-...",5,45,0.891725,392.301263,"[nan, 1.0842556401878616, 0.7752668638974063, ..."
...,...,...,...,...,...,...,...
14524,Avg_Chloro,"[Avg_Phaeo(t-1), Lingulodinium_polyedra(t-3), ...",5,9,-0.877208,143.681547,"[nan, 1.3339055417086112, 1.4614669666791014, ..."
28846,Avg_Chloro,"[Avg_Phaeo(t-1), Avg_Phaeo(t-5), Lingulodinium...",7,25,-0.878265,224.051685,"[nan, 0.7798990584761801, 1.0071487472127811, ..."
38160,Avg_Chloro,"[Akashiwo_sanguinea(t-1), Akashiwo_sanguinea(t...",9,9,-0.881096,143.816642,"[nan, 1.0791116401717775, 1.1478444810830941, ..."
15524,Avg_Chloro,"[Avg_Phaeo(t-1), Lingulodinium_polyedra(t-3), ...",5,15,-0.886389,206.409942,"[nan, 1.323666588434805, 1.4603379089335289, 2..."


In [ ]:

create_single_model(E,theta,target,embedding,lib, pred,HAB_embed_block,showPlot=False)